In [1]:
!pip install statsmodels


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy
from scipy import stats

%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['cm']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [30]:
data_plot = pd.read_csv('./TN-regplot_fit_data.csv')
data_exog = sm.add_constant(data_plot['LS']) #this is the x axis

In [31]:
data_exog

,const,LS
0,1.0,0.439128
1,1.0,0.437545
2,1.0,0.378273
3,1.0,0.437020
4,1.0,0.438599
...,...,...
102,1.0,0.437283
103,1.0,0.438599
104,1.0,0.306307
105,1.0,0.335388


In [32]:
rlm_model = sm.RLM(data_plot['pm_unc'], data_exog, 
                   M=sm.robust.norms.HuberT())

In [33]:
res= rlm_model.fit()

In [34]:
res.params

const    0.192134
LS      -0.065530
dtype: float64

In [35]:
rlm_model.weights

array([1.        , 1.        , 1.        , 1.        , 0.81072506,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.47867952, 1.        ,
       1.        , 1.        , 1.        , 0.75848158, 1.        ,
       1.        , 1.        , 0.73321365, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.43874173, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.58868222, 1.        ,
       0.44653014, 1.        , 1.        , 0.1327787 , 0.42551374,
       1.        , 0.59376275, 0.27126656, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.37614159,
       1.        , 0.5714151 , 1.        , 1.        , 1.        ,
       0.40890247, 0.65848218, 1.        , 0.44669241, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

In [ ]:
cm = plt.get_cmap('tab10')
params = {}
colors = np.array( [cm.colors[3],cm.colors[1],cm.colors[0]])
fig, axs = plt.subplots(2,2,figsize=(20,20))
for i,(ax, sysm, unit) in enumerate(zip(axs.flatten(),
                                        ['depth', 'astrom_prec', 'pm_unc', 'sigma_dT'],
                                  [r' $[ mag]$', r' $[arcsec]$', r' $[\frac{mas}{yr}]$', r' $[ d]$'])):
    
    params[sysm]  = pd.DataFrame(index = ['CI','LS','TPM'], columns = ['intercept','intercept_err' ,'slope', 'slope_err'])
    for j, (metric,color) in enumerate(zip(['CI','LS','TPM'], colors)):
            p=0
            print ("\t", j)
            x_fit = sm.add_constant(data_plot[sysm])
            fit_model = sm.RLM(data_plot[metric],  x_fit,
                               M=sm.robust.norms.HuberT()).fit()
            
            fit_mask = fit_model.weights
            #print (fit_mask)
            # you cannot remove the data - that gives you a different line fit 
            #X,Y = np.array(data_plot[sysm])[np.where(fit_mask>0.3)], np.array(data_plot[metric])[np.where(fit_mask>0.3)]
            #plot = pd.DataFrame({sysm:X, metric:Y})
            params[sysm]['slope'].iloc[j], params[sysm]['intercept'].iloc[j] = fit_model.params[sysm], fit_model.params['const']
            params[sysm]['slope_err'].iloc[j], params[sysm]['intercept_err'].iloc[j] =fit_model.bse[sysm], fit_model.bse['const']
            p = sns.regplot(x=data_plot[sysm], y=data_plot[metric], label=metric, 
                        robust=True, ax=ax, color=color, scatter_kws={"alpha":fit_mask})
            print(fit_model.params)
            #ax.scatter(data_plot[sysm], y=data_plot[metric], 
                        #alpha=fit_mask)
            
            
            #ax.plot(np.array(data_plot[sysm])[np.where(fit_mask<=0.3)], 
                    #np.array(data_plot[metric])[np.where(fit_mask<=0.3)],
                    #marker='X',markersize= 10 ,ls='', color=color)
    #plt.show()
    for j in range(3):
        x=p.get_lines()[j].get_xdata()
        y=p.get_lines()[j].get_ydata()
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
        #ax.plot(np.linspace(2,25), slope*np.linspace(2,25) +
                  # intercept, 'k-')
        print(slope, intercept)
            
    ax.set_ylabel(r'$PM_{MAF}$', fontsize=22) 
    ax.tick_params(axis='both', labelsize=18)
    
    if sysm =='sigma_dT':
        ax.set_xlabel(r'$\sigma_{dT}$'+unit , fontsize=22)
    elif sysm =='pm_unc':
        ax.set_xlabel(r'$\sigma_{\mu}$'+unit , fontsize=22)
        ax.set_xlim([data_plot[sysm].min() - 0.005,
                     0.2])
    else:
        ax.set_xlabel(sysm+unit , fontsize=22)
    ax.set_ylim([0.05,1.05])
    if i ==0:
        ax.legend(loc = 3, fontsize=18)
    
    
plt.savefig('./FoM-sysperf.pdf')

	 0
const   -20.915
depth     0.888
dtype: float64
	 1
const   -15.277
depth     0.644
dtype: float64
	 2
const   -4.082
depth    0.190
dtype: float64
0.8876213815315456 -20.915391725117736
0.6444712245507569 -15.277281976323232
0.1898634169385973 -4.082095389870829
	 0
const          2.594
astrom_prec   -1.809
dtype: float64
	 1
const          1.742
astrom_prec   -1.264
dtype: float64
	 2
const          0.913
astrom_prec   -0.355
dtype: float64
-1.8085425015113596 2.5938946852360685
-1.2641072504368567 1.7417043756544177
-0.3547526509293416 0.9131003691480977
	 0
const     0.582
pm_unc    0.704
dtype: float64
	 1
const     1.069
pm_unc   -4.000
dtype: float64
	 2
const     0.863
pm_unc   -1.962
dtype: float64
0.7042957000673218 0.5821108676969589
-4.00023639991698 1.068587424569186
-1.9621485329187023 0.8628763786229854
	 0


In [99]:
pd.set_option("display.precision", 3)
print(pd.concat(params).to_latex())


\begin{tabular}{llllll}
\toprule
         &     & intercept & intercept\_err &   slope & slope\_err \\
\midrule
depth & CI &     0.888 &         0.035 & -20.915 &     0.856 \\
         & LS &     0.644 &          0.05 & -15.277 &     1.211 \\
         & TPM &      0.19 &         0.027 &  -4.082 &     0.653 \\
astrom\_prec & CI &    -1.809 &         0.074 &   2.594 &     0.078 \\
         & LS &    -1.264 &         0.093 &   1.742 &     0.099 \\
         & TPM &    -0.355 &         0.052 &   0.913 &     0.056 \\
pm\_unc & CI &     0.704 &           0.2 &   0.582 &     0.034 \\
         & LS &      -4.0 &         0.178 &   1.069 &      0.03 \\
         & TPM &    -1.962 &         0.075 &   0.863 &     0.013 \\
sigma\_dT & CI &     0.004 &         0.002 &  -0.028 &     0.358 \\
         & LS &     0.004 &         0.003 &  -0.286 &     0.488 \\
         & TPM &     0.002 &         0.001 &   0.233 &     0.158 \\
\bottomrule
\end{tabular}



/var/folders/9p/2yw79snx27scqbt2xszxkfyc0000gp/T/ipykernel_28482/2180794676.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pd.concat(params).to_latex())


In [100]:
fit_model.bse

const       1.578e-01
sigma_dT    9.171e-04
dtype: float64